In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import scipy as sc

# Vote migration

In [2]:
v23=pd.read_csv('./vote23.csv',encoding='cp1255')
v22=pd.read_csv('./vote22.csv',encoding='cp1255')

In [3]:
v23=pd.read_csv('./vote24.csv',encoding='cp1255')
v22=pd.read_csv('./vote23.csv',encoding='cp1255')

In [4]:
v22['kalfi']=v22['קלפי'].astype(str)
v23['kalfi']=v23['קלפי'].astype(str)
v22['city']=v22['שם ישוב'].astype(str)
v23['city']=v23['שם ישוב'].astype(str)
cols = v23.columns.copy()
for cid in cols:
    v23 = v23.rename(columns={cid:cid+'_23'})
cols = v22.columns
for cid in cols:
    v22 = v22.rename(columns={cid:cid+'_22'})

In [5]:
xx=v23.merge(v22,how='inner',left_on=['city_23','kalfi_23'], right_on=['city_22', 'kalfi_22'])

In [6]:
xx

,סמל ועדה_23,ברזל_23,שם ישוב_23,סמל ישוב_23,קלפי_23,ריכוז_23,שופט_23,בזב_23,מצביעים_23,פסולים_23,...,ףז_22,ץ_22,ק_22,קי_22,קך_22,קץ_22,שס_22,Unnamed: 41_22,kalfi_22,city_22
0,6,3238,אבירים,1220,1.0,1,0,221,151,1,...,0,0,0,0,0,0,2,NaN,1.0,אבירים
1,5,1788,אדירים,113,1.0,1,0,234,180,0,...,0,0,0,0,0,0,58,NaN,1.0,אדירים
2,8,4113,אושה,278,1.0,1,0,382,276,0,...,0,0,0,0,0,0,0,NaN,1.0,אושה
3,17,10449,אחווה,1157,1.0,1,0,183,145,0,...,0,0,0,0,0,0,2,NaN,1.0,אחווה
4,17,10450,איבים,338,1.0,1,0,138,40,0,...,0,0,0,0,0,0,3,NaN,1.0,איבים
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4074,6,2571,יערה,795,1.0,2,0,569,385,9,...,0,0,0,0,0,0,91,NaN,1.0,יערה
4075,6,2590,נווה זיו,1314,1.0,1,0,566,418,0,...,0,0,0,0,0,1,15,NaN,1.0,נווה זיו
4076,6,3140,קורנית,1179,1.0,1,0,640,491,0,...,1,0,0,0,0,0,1,NaN,1.0,קורנית
4077,6,3143,שורשים,1235,1.0,1,0,423,290,0,...,1,0,0,0,0,0,1,NaN,1.0,שורשים


In [29]:
trans_23_22={}
trans_23_22['מחל'] = ['מחל']
trans_23_22['פה'] = ['פה']
trans_23_22['ודעם'] = ['ודעם']
trans_23_22['ג'] = ['ג']
trans_23_22['שס'] = ['שס']
trans_23_22['ל'] = ['ל']
trans_23_22[ 'אמת' ] = [ 'אמת' ]
trans_23_22[ 'ט' ] = [ 'נץ' ]
trans_23_22[ 'מרצ' ] = [ 'מרצ' ]
trans_23_22[ 'כן' ] = [ 'כן' ]
trans_23_22[ 'ת' ] = [ 'ת' ]
trans_23_22[ 'ב' ] = [ 'ב' ]
trans_23_22[ 'יז' ] = [ 'יז' ]
trans_23_22[ 'ר' ] = [ 'ר' ]
trans_23_22[ 'עם' ] = [ 'עם' ]
# trans_23_22[ '' ] = [ '' ]
# trans_23_22[ '' ] = [ '' ]


In [30]:
from collections import defaultdict
trans_22_23 = defaultdict(list)
for c23, c22 in trans_23_22.items():
    for cc22 in c22:
        trans_22_23[cc22].append(c23)

In [31]:
col_names = []
col_idx = {}
for idxa, a in enumerate(trans_23_22.keys()):
    cname = a+'_23'
    col_names.append(cname)
    col_idx[cname] = idxa
for idxb, a in enumerate(trans_22_23.keys()):
    cname = a+'_22'
    col_names.append(cname)
    col_idx[cname] = idxa+1+idxb

In [32]:
p22 = list(trans_22_23.keys())
p23 = list(trans_23_22.keys())
p22

['מחל',
 'פה',
 'ודעם',
 'ג',
 'שס',
 'ל',
 'אמת',
 'נץ',
 'מרצ',
 'כן',
 'ת',
 'ב',
 'יז',
 'ר',
 'עם']

In [33]:
p23

['מחל',
 'פה',
 'ודעם',
 'ג',
 'שס',
 'ל',
 'אמת',
 'ט',
 'מרצ',
 'כן',
 'ת',
 'ב',
 'יז',
 'ר',
 'עם']

## layout of variables:
22(0)->23(0)
22(1)->23(0)
...
22(end)->23(0)
22(abstained)->23(0)

So we have p22+1 source blocks each of p23+1 sinks

In [34]:
telaviv = v22['city_22'].value_counts().index[0]
xxx = xx[xx['city_22'] == 'בני ברק']
xxx = xx

In [35]:
telaviv

'תל אביב  יפו'

In [36]:
a=np.zeros([200000, (len(p22)+1) * (len(p23)+1) ])
b=np.zeros([200000])
crowidx = 0
for _,crow in xxx.iterrows():
#     if crow['בזב_22'] < 100:
#         continue
    tot_sink_voted = 0
    for idx_sink,csink in enumerate(p23):
        tot_voted=0
        for idx_source,csource in enumerate(p22):
            if csource+'_22' in crow:
                voted_source=crow[csource+'_22'] / crow['בזב_22']
            else:
                voted_source = 0
            a[crowidx,idx_source+idx_sink*(len(p22)+1)] = voted_source
            tot_voted += voted_source
        # add the non-voters -> sink column
        if tot_voted>1:
            tot_voted = 1
        a[crowidx, len(p22)+idx_sink*(len(p22)+1)] = 1-tot_voted
        sink_voted = crow[csink+'_23'] / crow['בזב_23']
        b[crowidx] = sink_voted
        tot_sink_voted += sink_voted
        crowidx += 1
    # add the sources->non-voters eq
    tot_voted = 0
    for idx_source,csource in enumerate(p22):
        if csource+'_22' in crow:
            voted_source=crow[csource+'_22'] / crow['בזב_22']
        else:
            voted_source = 0
        a[crowidx,idx_source+len(p23)*(len(p22)+1)] = voted_source
        tot_voted += voted_source
    # add the non-voters -> sink column
    if tot_voted>1:
        tot_voted = 1
    a[crowidx, len(p22)+len(p23)*(len(p22)+1)] = 1-tot_voted
    if tot_sink_voted > 1:
        tot_sink_voted = 1
    b[crowidx] = 1-tot_sink_voted
    crowidx += 1

In [37]:
# add the sum1 for all variables from a given source
for source_idx in range(len(p22)+1):
    for sink_idx in range(len(p23)+1):
        a[crowidx, source_idx + sink_idx*(len(p22)+1)] = 1000000
    b[crowidx] = 1000000
    crowidx += 1

In [38]:
a = a[:crowidx,:]
b = b[:crowidx]

In [39]:
a.shape

(65280, 256)

In [40]:
ff=np.linalg.lstsq(a,b,rcond=None)

In [41]:
import scipy.optimize
fff=scipy.optimize.nnls(a,b,maxiter=10000)

In [42]:
for idx_source, csource in enumerate(p22):
    for idx_sink, csink in enumerate(p23):
        print(csource+'-->'+csink+' : %f' % fff[0][idx_source+idx_sink*(len(p22)+1)])
    print(csource+'-->None : %f' % fff[0][idx_source+len(p23)*(len(p22)+1)])
    print()
for idx_sink, csink in enumerate(p23):
    print('None -->'+csink+' : %f' % fff[0][len(p23)+1+idx_sink*(len(p22)+1)])
print('None -->None : %f' % fff[0][len(p22)+(len(p23))*(len(p22)+1)])


מחל-->מחל : 0.759101
מחל-->פה : 0.010213
מחל-->ודעם : 0.000000
מחל-->ג : 0.000000
מחל-->שס : 0.008682
מחל-->ל : 0.008596
מחל-->אמת : 0.000000
מחל-->ט : 0.007819
מחל-->מרצ : 0.000000
מחל-->כן : 0.007132
מחל-->ת : 0.062451
מחל-->ב : 0.058734
מחל-->יז : 0.002635
מחל-->ר : 0.000000
מחל-->עם : 0.000000
מחל-->None : 0.074638

פה-->מחל : 0.000000
פה-->פה : 0.440782
פה-->ודעם : 0.000000
פה-->ג : 0.000000
פה-->שס : 0.000000
פה-->ל : 0.006272
פה-->אמת : 0.119469
פה-->ט : 0.000000
פה-->מרצ : 0.050078
פה-->כן : 0.232518
פה-->ת : 0.067232
פה-->ב : 0.013804
פה-->יז : 0.008545
פה-->ר : 0.000000
פה-->עם : 0.000000
פה-->None : 0.061300

ודעם-->מחל : 0.000000
ודעם-->פה : 0.000000
ודעם-->ודעם : 0.289564
ודעם-->ג : 0.000000
ודעם-->שס : 0.000000
ודעם-->ל : 0.000000
ודעם-->אמת : 0.000000
ודעם-->ט : 0.000000
ודעם-->מרצ : 0.000000
ודעם-->כן : 0.000000
ודעם-->ת : 0.000000
ודעם-->ב : 0.000000
ודעם-->יז : 0.000000
ודעם-->ר : 0.000000
ודעם-->עם : 0.241763
ודעם-->None : 0.468672

ג-->מחל : 0.000000
ג-->פה : 0.0000

In [308]:
v23['city_23'].value_counts()

תל אביב  יפו    680
ירושלים         666
חיפה            405
ראשון לציון     339
פתח תקווה       305
               ... 
תאשור             1
גליל ים           1
מרחב עם           1
רמת מגשימים       1
צופר              1
Name: city_23, Length: 1211, dtype: int64